In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install autokeras

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}

from numpy import mean
from numpy import std
import numpy as np
from matplotlib import pyplot
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat
import numpy as np
import PIL
import cv2
from sklearn.model_selection import train_test_split
import autokeras as ak
import tensorflow
import warnings

In [ ]:
import numpy as np
#x = np.load('/content/drive/MyDrive/PCG_signal_time_frequency_image/github_database_5Class_complete/classification_scheme1/scheme1_x.npy')
#y = np.load('/content/drive/MyDrive/PCG_signal_time_frequency_image/github_database_5Class_complete/classification_scheme1/scheme1_y.npy')

x = np.load('/content/drive/MyDrive/PCG_signal_time_frequency_image/github_database_5Class_complete/classification_scheme2/scheme2_x.npy')
y = np.load('/content/drive/MyDrive/PCG_signal_time_frequency_image/github_database_5Class_complete/classification_scheme2/scheme2_y.npy')

In [ ]:
test_acc_list = []
K_cappa_list = []
precision_list = []
recall_list = []
F1_list = []

for iter in range(5):
  print("Trial Number : ", (iter+1))
  loaded_model = tensorflow.keras.models.load_model("/content/drive/MyDrive/PCG_signal_time_frequency_image/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)


  y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],5))

  for i in range(np.array(y_train).shape[0]):
    label = y_train[i]
    y_tr_one_hot[i][int(label)] = 1
  K.clear_session()
  
#clf = ak.ImageClassifier(overwrite=True, max_trials=4)
#clf.fit(x_train, y_tr_one_hot,epochs=40)
#results = clf.predict(x_test)
#model = clf.export_model()
#model.save("/content/drive/MyDrive/PCG_signal_time_frequency_image/model_autokeras", save_format="tf")
#print(model.summary())

  optimizer = keras.optimizers.Adam(lr=0.001)
  loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  y_val_one_hot  = np.zeros((np.array(y_val).shape[0],5))

  for i in range(np.array(y_val).shape[0]):
    label = y_val[i]
    y_val_one_hot[i][int(label)] = 1

  hist = loaded_model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot),batch_size=128, epochs=10,verbose=0)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],5))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = loaded_model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  test_acc_list.append(test_acc)

  ##Evaluating Sensitivity, Accuracy and Kappa scores
  y_prob = loaded_model.predict(x_test) 
  Y_pred = y_prob.argmax(axis=-1)

  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  print("cohen kappa scores:" ,K_cappa)
  K_cappa_list.append(K_cappa)

  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)

  precision = sklearn.metrics.precision_score(y_test,Y_pred,average='micro')
  print('precision : ', precision)
  precision_list.append(precision)

  recall = sklearn.metrics.recall_score(y_test,Y_pred,average='micro')
  print('recall : ', recall)
  recall_list.append(recall)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  print("F1 : ", F1)
  F1_list.append(F1)


print('5-Trial Accuracy:',sum(test_acc_list)/len(test_acc_list))
print("5-Trial cohen kappa scores:" ,sum(K_cappa_list)/len(K_cappa_list))
print('5-Trial precision : ', sum(precision_list)/len(precision_list))
print('5-Trial recall : ', sum(recall_list)/len(recall_list))
print("5-Trial F1 : ", sum(F1_list)/len(F1_list))

In [ ]:
test_acc_list = []
K_cappa_list = []
precision_list = []
recall_list = []
F1_list = []

for iter in range(5):
  print("Trial Number : ", (iter+1))
  train_ratio=0.70
  validation_ratio=0.10
  test_ratio=0.20
  loaded_model = tensorflow.keras.models.load_model("/content/drive/MyDrive/PCG_signal_time_frequency_image/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=1)
  x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=1)


  y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],5))

  for i in range(np.array(y_train).shape[0]):
    label = y_train[i]
    y_tr_one_hot[i][int(label)] = 1

  K.clear_session()
  #modelf = model_define()

  #print(modelf.summary())
  optimizer = keras.optimizers.Adam(lr=0.001)
  loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  y_val_one_hot  = np.zeros((np.array(y_val).shape[0],5))

  for i in range(np.array(y_val).shape[0]):
    label = y_val[i]
    y_val_one_hot[i][int(label)] = 1

  hist = loaded_model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot),batch_size=128, epochs=10,verbose=0)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],5))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = loaded_model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  test_acc_list.append(test_acc)

  ##Evaluating Sensitivity, Accuracy and Kappa scores
  y_prob = loaded_model.predict(x_test) 
  Y_pred = y_prob.argmax(axis=-1)

  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  print("cohen kappa scores:" ,K_cappa)
  K_cappa_list.append(K_cappa)

  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)

  precision = sklearn.metrics.precision_score(y_test,Y_pred,average='micro')
  print('precision : ', precision)
  precision_list.append(precision)

  recall = sklearn.metrics.recall_score(y_test,Y_pred,average='micro')
  print('recall : ', recall)
  recall_list.append(recall)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  print("F1 : ", F1)
  F1_list.append(F1)


print('5-Trial Accuracy:',sum(test_acc_list)/len(test_acc_list))
print("5-Trial cohen kappa scores:" ,sum(K_cappa_list)/len(K_cappa_list))
print('5-Trial precision : ', sum(precision_list)/len(precision_list))
print('5-Trial recall : ', sum(recall_list)/len(recall_list))
print("5-Trial F1 : ", sum(F1_list)/len(F1_list))

In [ ]:
test_acc_list = []
K_cappa_list = []
precision_list = []
recall_list = []
F1_list = []

for iter in range(5):
  print("Trial Number : ", (iter+1))
  train_ratio=0.60
  validation_ratio=0.20
  test_ratio=0.20
  loaded_model = tensorflow.keras.models.load_model("/content/drive/MyDrive/PCG_signal_time_frequency_image/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=1)
  x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=1)


  y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],5))

  for i in range(np.array(y_train).shape[0]):
    label = y_train[i]
    y_tr_one_hot[i][int(label)] = 1

  K.clear_session()
  #modelf = model_define()

  #print(modelf.summary())
  optimizer = keras.optimizers.Adam(lr=0.001)
  loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  y_val_one_hot  = np.zeros((np.array(y_val).shape[0],5))

  for i in range(np.array(y_val).shape[0]):
    label = y_val[i]
    y_val_one_hot[i][int(label)] = 1

  hist = loaded_model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot),batch_size=128, epochs=10,verbose=0)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],5))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = loaded_model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  test_acc_list.append(test_acc)

  ##Evaluating Sensitivity, Accuracy and Kappa scores
  y_prob = loaded_model.predict(x_test) 
  Y_pred = y_prob.argmax(axis=-1)

  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  print("cohen kappa scores:" ,K_cappa)
  K_cappa_list.append(K_cappa)

  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)

  precision = sklearn.metrics.precision_score(y_test,Y_pred,average='micro')
  print('precision : ', precision)
  precision_list.append(precision)

  recall = sklearn.metrics.recall_score(y_test,Y_pred,average='micro')
  print('recall : ', recall)
  recall_list.append(recall)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  print("F1 : ", F1)
  F1_list.append(F1)


print('5-Trial Accuracy:',sum(test_acc_list)/len(test_acc_list))
print("5-Trial cohen kappa scores:" ,sum(K_cappa_list)/len(K_cappa_list))
print('5-Trial precision : ', sum(precision_list)/len(precision_list))
print('5-Trial recall : ', sum(recall_list)/len(recall_list))
print("5-Trial F1 : ", sum(F1_list)/len(F1_list))

In [ ]:
test_acc_list = []
K_cappa_list = []
precision_list = []
recall_list = []
F1_list = []

for iter in range(5):
  print("Trial Number : ", (iter+1))
  train_ratio=0.75
  validation_ratio=0.15
  test_ratio=0.20
  loaded_model = tensorflow.keras.models.load_model("/content/drive/MyDrive/PCG_signal_time_frequency_image/model_autokeras", custom_objects=ak.CUSTOM_OBJECTS)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=1)
  x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio+validation_ratio), random_state=1)


  y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],5))

  for i in range(np.array(y_train).shape[0]):
    label = y_train[i]
    y_tr_one_hot[i][int(label)] = 1

  K.clear_session()
  #modelf = model_define()

  #print(modelf.summary())
  optimizer = keras.optimizers.Adam(lr=0.001)
  loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  y_val_one_hot  = np.zeros((np.array(y_val).shape[0],5))

  for i in range(np.array(y_val).shape[0]):
    label = y_val[i]
    y_val_one_hot[i][int(label)] = 1

  hist = loaded_model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot),batch_size=128, epochs=10,verbose=0)

  y_te_one_hot  = np.zeros((np.array(y_test).shape[0],5))

  for i in range(np.array(y_test).shape[0]):
    label = y_test[i]
    y_te_one_hot[i][int(label)] = 1

  test_loss, test_acc = loaded_model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
  print('Accuracy:',test_acc)
  test_acc_list.append(test_acc)

  ##Evaluating Sensitivity, Accuracy and Kappa scores
  y_prob = loaded_model.predict(x_test) 
  Y_pred = y_prob.argmax(axis=-1)

  K_cappa = sklearn.metrics.cohen_kappa_score(y_test,Y_pred)
  print("cohen kappa scores:" ,K_cappa)
  K_cappa_list.append(K_cappa)

  cm1 = confusion_matrix(y_test,Y_pred)
  print("confusion matrix \n",cm1)

  precision = sklearn.metrics.precision_score(y_test,Y_pred,average='micro')
  print('precision : ', precision)
  precision_list.append(precision)

  recall = sklearn.metrics.recall_score(y_test,Y_pred,average='micro')
  print('recall : ', recall)
  recall_list.append(recall)

  F1 = sklearn.metrics.f1_score(y_test,Y_pred,average="micro")
  print("F1 : ", F1)
  F1_list.append(F1)


print('5-Trial Accuracy:',sum(test_acc_list)/len(test_acc_list))
print("5-Trial cohen kappa scores:" ,sum(K_cappa_list)/len(K_cappa_list))
print('5-Trial precision : ', sum(precision_list)/len(precision_list))
print('5-Trial recall : ', sum(recall_list)/len(recall_list))
print("5-Trial F1 : ", sum(F1_list)/len(F1_list))